In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from tensorflow import keras
model = keras.models.load_model('/media/tohn/SSD/ModelTrainByImages/R2_Full/models/B5R4_15AB_Full_MultiLabel_fold1_1')
height = width = model.input_shape[1]

In [3]:
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image/'#แก้
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Testdf.csv')#แก้
validation_dir = os.path.join(base_dir, 'test')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf.csv')#แก้
base_dir0 = '/media/tohn/SSD/Images/Image/'#แก้
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [4]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Testdf.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(1312, 22)
(1312, 22)
Normal:  (857, 22)
Abnormal:  (455, 22)


,Unnamed: 0,Unnamed: 0.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,Views,...,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,filename
0,111,111,40,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,86.0,133.0,477.0,311.0,0.092664,0.148873,0.513514,0.346614,AB01 P1 C040.JPG
1,112,112,40,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,163.0,139.0,532.0,385.0,0.171698,0.154849,0.560377,0.428287,AB01 P2 C040.JPG
2,113,113,40,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,...,https://irisprodseatraining.blob.core.windows....,127.0,135.0,605.0,640.0,0.140316,0.150865,0.667984,0.711155,AB01 P4-1 C040.JPG
3,114,114,40,P5,P51,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-C,...,https://irisprodseatraining.blob.core.windows....,59.0,96.0,643.0,587.0,0.063462,0.107041,0.690385,0.653386,AB01 P5-1 C040.JPG
4,115,115,40,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,...,https://irisprodseatraining.blob.core.windows....,199.0,132.0,618.0,562.0,0.216374,0.146881,0.672515,0.625498,AB01 P3-1 C040.JPG


In [5]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Sub_class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 1312 validated image filenames belonging to 15 classes.
{0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB04', 4: 'AB05', 5: 'AB06', 6: 'AB07', 7: 'AB081', 8: 'AB082', 9: 'AB083', 10: 'AB09', 11: 'AB10', 12: 'AB11', 13: 'AB12', 14: 'Normal'}


In [6]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model.predict([x])
    
    return result[0]

In [7]:
# #Predict
# pred_list = list()
# prob_list = list()
# img_path=dataframe['Path Crop'].tolist()

# for i in range(0,len(img_path)):
#     predict = predict_image(img_path[i])
#     result = np.where(predict > 0.5)
#     print(result)
#     pred = []
#     if len(result[0]) ==0:
#         pred = []
#         pred.append('Normal')
#         pred_list.append(pred)
#     else:
#         pred = []
#         for i in range(len(result[0])):
#             pred.append(labels[result[0][i]])
#         pred_list.append(pred)

In [8]:
img_path=dataframe['Path Crop'].tolist()

In [9]:
pred_list= []
for i in range(0,len(img_path)):
    predict = predict_image(img_path[i]) 
    pred_list.append(predict)

In [10]:
pred_list[0]

array([1.38248990e-09, 6.12178177e-04, 2.62050391e-13, 2.76290746e-14,
       1.81558642e-12, 8.25019249e-16, 1.00088645e-13, 1.72272407e-12,
       6.67043740e-16, 3.66341803e-16, 8.64751115e-09, 5.19167352e-16,
       3.27565931e-16, 9.55967835e-13], dtype=float32)

In [11]:
# threhold = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.80,0.9,1]
# Pred_list_ =[]
# for j in threhold:
#     Pred_list =[]
#     for i in pred_list:
#         result = np.where(i > j)
#         print(result)

In [12]:
threhold = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
Pred_list_ =[]
for j in threhold:
    for i in pred_list:
        Pred_list =[]
        result = np.where(i > j)
        pred = []
        if len(result[0]) ==0:
            pred = []
            pred.append('Normal')
            Pred_list.append(pred)
        else:
            pred = []
            for m in range(len(result[0])):
                pred.append(labels[result[0][m]])
            Pred_list.append(pred)
        Pred_list_.append(Pred_list)

In [13]:
pred0 = Pred_list_[0:1312]
pred01 = Pred_list_[1312:1312*2]
pred02 = Pred_list_[1312*2:1312*3]
pred03 = Pred_list_[1312*3:1312*4]
pred04 = Pred_list_[1312*4:1312*5]
pred05 = Pred_list_[1312*5:1312*6]
pred06 = Pred_list_[1312*6:1312*7]
pred07 = Pred_list_[1312*7:1312*8]
pred08 = Pred_list_[1312*8:1312*9]
pred09 = Pred_list_[1312*9:1312*10]
pred10 = Pred_list_[1312*10:1312*11]

In [14]:
Sub_class = dataframe['Sub_class'].tolist()
aa=[]
for i in Sub_class:
    a =[]
    a.append(i)
    aa.append(a)

In [15]:
col_name = ['Act_label']
dff2 = pd.DataFrame(columns = col_name)

In [16]:
pred01[0][0]

['Normal']

In [17]:
# for i in pred01

In [18]:
dff2['Act_label'] = aa
dff2['Pred_label01'] = pred01
dff2['Pred_label02'] = pred02
dff2['Pred_label03'] = pred03
dff2['Pred_label04'] = pred04
dff2['Pred_label05'] = pred05
dff2['Pred_label06'] = pred06
dff2['Pred_label07'] = pred07
dff2['Pred_label08'] = pred08
dff2['Pred_label09'] = pred09
dff2['Pred_label10'] = pred10
dff2

,Act_label,Pred_label01,Pred_label02,Pred_label03,Pred_label04,Pred_label05,Pred_label06,Pred_label07,Pred_label08,Pred_label09,Pred_label10
0,[AB01],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]]
1,[AB01],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]]
2,[AB01],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]]
3,[AB01],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]]
4,[AB01],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]]
...,...,...,...,...,...,...,...,...,...,...,...
1307,[Normal],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]]
1308,[Normal],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]]
1309,[Normal],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]]
1310,[Normal],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]],[[Normal]]


# Evaluation

In [19]:
Act = dff2['Act_label'].tolist()
# Pred = dff2['Pred_label01'].tolist()
# Pred = dff2['Pred_label02'].tolist()
# Pred = dff2['Pred_label03'].tolist()
# Pred = dff2['Pred_label04'].tolist()
# Pred = dff2['Pred_label05'].tolist()
# Pred = dff2['Pred_label06'].tolist()
# Pred = dff2['Pred_label07'].tolist()
# Pred = dff2['Pred_label08'].tolist()
Pred = dff2['Pred_label09'].tolist()
# Pred = dff2['Pred_label10'].tolist()

In [20]:
class_ = ['AB01', 'AB02','AB03','AB04','AB05','AB06','AB07','AB081','AB082','AB083','AB09','AB10','AB11','AB12','Normal']
cla_act = []
for i in Act:
    if len(list(i)) == 1:
        a = str(list(i)[0])
        for j in range(len(class_)):
            if a == class_[j]:
                m = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                m[j] = 1
                cla_act.append(m)
    else:
        m = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        for j in list(i):
            for k in range(len(class_)):
                if j == class_[k]:
                    m[k] = 1
        cla_act.append(m)

In [21]:
class_ = ['AB01', 'AB02','AB03','AB04','AB05','AB06','AB07','AB081','AB082','AB083','AB09','AB10','AB11','AB12','Normal']
cla_pred = []
for i in Pred:
    if len(list(i[0])) == 1:
        a = str(list(i[0])[0])
        for j in range(len(class_)):
            if a == class_[j]:
                m = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                m[j] = 1
                cla_pred.append(m)
    else:
        m = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        for j in list(i[0]):
            for k in range(len(class_)):
                if j == class_[k]:
                    m[k] = 1
        cla_pred.append(m)

In [22]:
# len(cla_pred)

In [23]:
import numpy as np
y_true = np.array(cla_act)
y_pred = np.array(cla_pred)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_true, y_pred)

0.6524390243902439


array([[[1237,    1],
        [  74,    0]],

       [[1247,    5],
        [  59,    1]],

       [[1294,    0],
        [  18,    0]],

       [[1269,    0],
        [  43,    0]],

       [[1282,    1],
        [  26,    3]],

       [[1291,    0],
        [  21,    0]],

       [[1291,    0],
        [  21,    0]],

       [[1280,    0],
        [  32,    0]],

       [[1284,    0],
        [  28,    0]],

       [[1301,    0],
        [  11,    0]],

       [[1285,    1],
        [  26,    0]],

       [[1302,    0],
        [  10,    0]],

       [[1289,    0],
        [  23,    0]],

       [[1248,    5],
        [  56,    3]],

       [[  12,  443],
        [   8,  849]]])

In [24]:
from sklearn.metrics import classification_report
label_names = ['AB01', 'AB02','AB03','AB04','AB05','AB06','AB07','AB081','AB082','AB083','AB09','AB10','AB11','AB12','Normal']
print(classification_report(y_true, y_pred,target_names=label_names))

              precision    recall  f1-score   support

        AB01       0.00      0.00      0.00        74
        AB02       0.17      0.02      0.03        60
        AB03       0.00      0.00      0.00        18
        AB04       0.00      0.00      0.00        43
        AB05       0.75      0.10      0.18        29
        AB06       0.00      0.00      0.00        21
        AB07       0.00      0.00      0.00        21
       AB081       0.00      0.00      0.00        32
       AB082       0.00      0.00      0.00        28
       AB083       0.00      0.00      0.00        11
        AB09       0.00      0.00      0.00        26
        AB10       0.00      0.00      0.00        10
        AB11       0.00      0.00      0.00        23
        AB12       0.38      0.05      0.09        59
      Normal       0.66      0.99      0.79       857

   micro avg       0.65      0.65      0.65      1312
   macro avg       0.13      0.08      0.07      1312
weighted avg       0.47   

/home/yupaporn/miniconda3/envs/USAI3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
Pred_list =[]
for i in pred_list:
    result = np.where(i > 0.5)
    pred = []
    if len(result[0]) ==0:
        pred = []
        pred.append('Normal')
        Pred_list.append(pred)
    else:
        pred = []
        for i in range(len(result[0])):
            pred.append(labels[result[0][i]])
        Pred_list.append(pred)

In [26]:
Pred_list

[['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['AB01'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['AB01',
  'AB02',
  'AB03',
  'AB04',
  'AB05',
  'AB06',
  'AB07',
  'AB081',
  'AB082',
  'AB083',
  'AB09',
  'AB10',
  'AB11',
  'AB12'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['AB12'],
 ['AB01'],
 ['Normal'],
 ['AB01'],
 ['Normal'],
 ['AB01',
  'AB02',
  'AB03',
  'AB04',
  'AB05',
  'AB06',
  'AB07',
  'AB081',
  'AB082',
  'AB083',
  'AB09',
  'AB10',
  'AB11',
  'AB12'],
 ['Normal'],
 ['AB12'],
 ['AB01'],
 ['AB01'],
 ['AB01'],
 ['AB01'],
 ['AB01'],
 ['Normal'],
 ['AB01'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Normal'],
 ['Nor

In [27]:
result = np.where(predict > 0.5)

pred = []
if len(result[0]) ==0:
    pred = []
    pred.append('Normal')
    pred_list.append(pred)
else:
    pred = []
    for i in range(len(result[0])):
        pred.append(labels[result[0][i]])
    pred_list.append(pred)

In [28]:
pred_list

[array([1.2637738e-11, 3.5238187e-07, 3.4332401e-09, 2.7886142e-12,
        6.8580276e-13, 7.5093051e-13, 6.5039857e-12, 9.1096638e-14,
        1.5883884e-13, 1.8597162e-12, 4.3820501e-14, 4.1288083e-13,
        3.5772685e-14, 8.6167930e-08], dtype=float32),
 array([9.0441148e-11, 3.3852357e-06, 1.6336993e-18, 1.7016899e-19,
        1.4871725e-16, 9.3489031e-17, 5.8236549e-16, 6.5597044e-18,
        3.3847860e-17, 8.6234942e-17, 3.0520708e-18, 1.2321295e-15,
        8.7119747e-17, 2.9832206e-09], dtype=float32),
 array([1.5695119e-05, 4.3634987e-15, 9.7456815e-24, 2.2153646e-24,
        1.3269151e-21, 3.2399931e-23, 2.0962273e-22, 1.9369001e-19,
        1.2156693e-20, 4.5740427e-22, 1.8027696e-24, 2.2410791e-22,
        6.9921570e-24, 2.7201186e-22], dtype=float32),
 array([8.7813874e-05, 1.4919709e-13, 1.2967132e-18, 4.7753644e-20,
        3.5836998e-19, 1.2239672e-18, 2.0254683e-19, 1.0200602e-18,
        2.8940332e-18, 1.4632341e-16, 3.3531841e-17, 2.6410045e-18,
        1.1844460e-

In [29]:
#Predict
pred_list = list()
prob_list = list()
img_path=dataframe['Path Crop'].tolist()

for i in range(0,len(img_path)):
    predict = predict_image(img_path[i])
    result = np.where(predict > 0.5)
    print(result)
    pred = []
    if len(result[0]) ==0:
        pred = []
        pred.append('Normal')
        pred_list.append(pred)
    else:
        pred = []
        for i in range(len(result[0])):
            pred.append(labels[result[0][i]])
        pred_list.append(pred)

(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([0]),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([13]),)
(array([0]),)
(array([], dtype=int64),)
(array([0]),)
(array([], dtype=int64),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([], dtype=int64),)
(array([13]),)
(array([0]),)
(array([0]),)
(array([0]),)
(array([0]),)
(array([0]),)
(array([], dtype=int64),)
(array([0]),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([

(array([], dtype=int64),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([4]),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([0]),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([0]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([], dtype=int64),)
(array([0]),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([4]),)
(array([], dtype=int64),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([0]),)
(array([0]),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([10]),)
(array([0]),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([0]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 

(array([0]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([0]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([0, 1, 2]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([4]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)


(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 2, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9,

KeyboardInterrupt: 

In [ ]:
pred_list

In [ ]:
result[0]

In [ ]:
for i in range(len(result[0])):
    print(labels[result[0][i]])

In [ ]:
predict[predict > 0.5]

In [ ]:
#Predict
pred_list = list()
prob_list = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    predict = predict_image(img_path[i])
    result = np.argmax(predict)
    pred_list.append(labels[result])
    prob_list.append(predict[result])

dataframe['category'] = pred_list
dataframe['Prob'] = prob_list